## 导入库

In [1]:
import magpylib as magpy
import numpy as np
import pyvista as pv
from scipy.spatial.transform import Rotation as R
import itertools
import matplotlib.pyplot as plt
from PIL import Image
import os
import pandas as pd

## 构建类与方法

In [2]:
class My_Magic:
    """
    类的成员包括：属性和方法
    属性分为：静态和实例
    方法：普通、类、静态
    
    """
    magnetlist = []
    def __init__(self):
        super().__init__()
        self.sensor_position = [50, 50, 0, 10, 10, 10, -10]
        self.magnet_position = [3, 3, 3, 1, 1, 1, 30, 30, 30]
        # 在此构建静态属性-对象，主要指向磁铁类型
        #  Create some magnet with magnetic polarization
        self.sensor_list = []
        self.space_euler_list = []
        self.magnet_objects =   magpy.magnet.Cuboid(magnetization=(0, 0, 1000), # y轴的负半轴 100 mT
                                    dimension=(30, 10, 20),        # 大小为mm
                                    position=(10, 10, 10),        # 位置
                                    orientation = R.from_rotvec((0,0,0), degrees=True)
                                    )

                # magpy.magnet.Cuboid(magnetization=(1000, 0, 0), # y轴的负半轴 100 mT
                #                     dimension=(3, 1, 2),        # 大小为mm
                #                     position=(1, 0, 0),        # 位置
                #                     orientation = R.from_rotvec((0,0,-45), degrees=True)
                #                     ),
        self.sensor_name = []
        self.magnet_name = []
        self.sensor_data = []
        self.magpos_data = []
        self.cnn_sensor_data = []
        self.cnn_magpos_data = []
        self.df = pd.DataFrame()
        self.sensor_imag_data = []

    
    def sensor_generate(self,sensor_position):
        """
        sensor array
        """
        X, Y, Z, xn, yn, zn, z0= sensor_position
        x_range = range(0, X, xn)
        y_range = range(0, Y, yn)
        z_range = range(z0, Z, zn)
        self.sensor_name_generate(x_range,y_range,z_range)
        for x, y, z in itertools.product(x_range, y_range, z_range):
            self.sensor_list.append(magpy.Sensor(pixel=(0, 0, 0), position=(x, y, z)))
        return self.sensor_list
    
    def space_euler_position(self,magnet_position):
        """
        magic posture
        """
        X, Y, Z , xn, yn, zn, ro_n, pi_n, ya_n = self.magnet_position
        # 定义要遍历的空间范围和步长
        x_range = range(0, X, xn)
        y_range = range(0, Y, yn)
        z_range = range(0, Z, zn)
        # 定义要遍历的角度范围和步长
        roll_range  = range(0, 360, ro_n)
        pitch_range = range(0, 360, pi_n)
        yaw_range   = range(0, 360, ya_n)
        space_euler_list = []
        for x, y, z, roll, pitch, yaw in itertools.product(x_range, y_range, z_range, roll_range, pitch_range, yaw_range):
            space_euler_list.append([x, y, z, roll, pitch, yaw]) 
        self.space_euler_list = np.array(space_euler_list)
        return self.space_euler_list
    def sensor_name_generate(self,x_range,y_range,z_range):
        name_list = []
        for i in range(len(x_range)):
            for j in range(len(y_range)):
                for k in range(len(z_range)):
                    name_list.append("sensor"+str(i)+str(j)+str(k) +"x")
                    name_list.append("sensor"+str(i)+str(j)+str(k) +"y")
                    name_list.append("sensor"+str(i)+str(j)+str(k) +"z")

        for i in range(len(name_list)):
            self.sensor_name.append(name_list[i])
        return self.sensor_name
    
    def just_magnet(self,space_euler_list):
        self.magnet_objects.orientation = R.from_rotvec(list(space_euler_list[:,0:3]), degrees=True)
        self.magnet_objects.position = list(space_euler_list[:,3:7])
        return self.magnet_objects

    def sensor_data_showimage1(self,matrix,img_ctl_str= "openimg",channels = 3,gif_ctl_str = "savegif",gif_save_path = "",int_ctl_str = "returnint"):
        """
        This function generates a dictionary of sensor objects, with the key being the sensor name.
        """
        max_value = np.max(np.abs(matrix))
        min_value = -max_value
        # 进行最小-最大归一化
        normalized_data = (matrix - min_value) *255 / (max_value - min_value)
        ## create images and compute frame_n and frame_img_mm
        images = []
        frame_n = np.shape(normalized_data)[0]
        frame_cnt = 0
        frame_img_mm = np.sqrt(np.shape(normalized_data)[1]/3).astype(int)
        matrix_Data = np.zeros((frame_n,frame_img_mm,frame_img_mm,channels))
        red_channel = []
        green_channel = []
        blue_channel = []
        for frame_data in normalized_data :

            if(channels == 3):
                red_channel = frame_data[::3]
                green_channel = frame_data[1::3]
                blue_channel = frame_data[2::3]
            else:
                red_channel = frame_data[:]
            
            # 创建一个6x6的RGB图像
            image = Image.new("RGB", (frame_img_mm, frame_img_mm))
            # 填充图像
            for x in range(frame_img_mm):
                for y in range(frame_img_mm):
                    pixel_index = y * frame_img_mm + x
                    image.putpixel((x, y), (red_channel[pixel_index].astype(int), green_channel[pixel_index].astype(int), blue_channel[pixel_index].astype(int)))
                    matrix_Data[frame_cnt,x,y] = (red_channel[pixel_index], green_channel[pixel_index], blue_channel[pixel_index])
            # image.save("output.png")
            images.append(image)
            if img_ctl_str == "openimg":
                plt.imshow(image)
                plt.axis('off')  # 隐藏坐标轴
                plt.show()
            frame_cnt = frame_cnt + 1 # frame cnt
        if(gif_ctl_str == "savegif"):
            output_gif = gif_save_path + '\\output.gif'  # 保存的 GIF 文件名
            images[0].save(output_gif, save_all=True, append_images=images[1:], duration=100, loop=0)
        if(int_ctl_str == "returnimg"):
            return images
        else:
            return matrix_Data

## 构建磁铁姿态位置和传感器阵列

In [3]:
My_Magic_u = My_Magic()
My_Magic_u.sensor_position  = [600, 600, 0, 100, 100, 100, -100]
My_Magic_u.sensor_list      = My_Magic_u.sensor_generate(My_Magic_u.sensor_position)
My_Magic_u.magnet_position  = [600, 600, 200, 100, 100, 50, 90, 90, 90] # 6*6*4*4*4
My_Magic_u.space_euler_list = My_Magic_u.space_euler_position(My_Magic_u.magnet_position)
My_Magic_u.magnet_objects.orientation = R.from_rotvec(My_Magic_u.space_euler_list[:,3:7] , degrees=True)
My_Magic_u.magnet_objects.position =  My_Magic_u.space_euler_list[:,0:3]

# print("磁铁空间位置：", My_Magic_u.magnet_position)
print("磁铁空间位置向量大小：",np.shape(My_Magic_u.space_euler_list))
# print("传感器空间位置：", My_Magic_u.sensor_position )
# print("传感器名字列表大小：", np.shape(My_Magic_u.sensor_name))
magpy.show(My_Magic_u.magnet_objects,My_Magic_u.sensor_list, backend='plotly')

磁铁空间位置向量大小： (9216, 6)


## 计算磁场大小

In [4]:
H = magpy.getH(My_Magic_u.magnet_objects, My_Magic_u.sensor_list)
print("产生磁场大小:",np.shape(H[0]))
H_just = np.zeros((np.shape(H)[0],np.shape(H)[1] * np.shape(H)[2]))
for i in range(0,np.shape(H)[1]):
    # print((i + 1)*3 -3 ,(i+1)*3)
    H_just[:,((i + 1)*3 -3) : ((i+1)*3)] = H[:,i]
print("magnetpose与sensor数量:",np.shape(H_just))


产生磁场大小: (36, 3)
magnetpose与sensor数量: (9216, 108)


## 保存数据

In [8]:
My_Magic_u.magnet_name = ['x(mm)','y(mm)','z(mm)','X_theta(deg)','Y_theta(deg)','Z_theta(deg)']
column = My_Magic_u.magnet_name + My_Magic_u.sensor_name
print("磁铁位置与传感器位置和输出数据标签：",len(column))

data_list = np.concatenate((np.round(My_Magic_u.space_euler_list), H_just), axis=1)
print("传感器输出数据：",np.shape(data_list))
My_Magic_u.df = pd.DataFrame(data_list, columns=column)
data_path = "../../data/magpy_data/"
My_Magic_u.df.to_csv(data_path + "mag_sensor_data.csv", index=False)


磁铁位置与传感器位置和输出数据标签： 114
传感器输出数据： (9216, 114)


## 构建训练模型

### 明确输入输出

In [9]:
### 输入为阵列传感器的输出 ->(4096, 16, 3) 4096个位置姿态。16个sensor， 3个方向
My_Magic_u.sensor_data = np.array(My_Magic_u.df[My_Magic_u.sensor_name]) # type: ignore
print("多姿态传感器的输出矩阵：",np.shape(My_Magic_u.sensor_data))
print("多姿态传感器的输出矩阵方差：",np.var(My_Magic_u.sensor_data))
My_Magic_u.magpos_data = np.array(My_Magic_u.df[My_Magic_u.magnet_name]) # type: ignore

My_Magic_u.sensor_imag_data = My_Magic_u.sensor_data_showimage1(My_Magic_u.sensor_data,img_ctl_str ="closeimg",gif_save_path = data_path, gif_ctl_str = "savegif",int_ctl_str="returnimg") # type: ignore
print("sensor数据帧:",np.shape(My_Magic_u.sensor_imag_data))

多姿态传感器的输出矩阵： (9216, 108)
多姿态传感器的输出矩阵方差： 0.0015418675705863628
sensor数据帧: (9216, 6, 6, 3)
